In [1]:
"""
todo
三种方法：
1 gensim 训练　embedding
2 openKE 训练　embedding
3 openKE　分词训练　embedding　
"""

'\ntodo\n三种方法：\n1 gensim 训练\u3000embedding\n2 openKE 训练\u3000embedding\n3 openKE\u3000分词训练\u3000embedding\u3000\n'

In [2]:
# coding:u8
import json
import numpy as np
from collections import defaultdict
from tqdm import tqdm as tqdm
from gensim.models import word2vec
import jieba

In [3]:
# 读取知识库
kb_data = []
kb = {}
with open('data/raw_data/kb_data','r') as f:
    for line in f:
        item = eval(str(json.loads(line)).lower())
        kb[item['subject_id']] = item
        kb_data.append(item)


In [4]:
# OpenKE 默认方法

In [5]:
predicate = {}
subject = {}
type_object = {}

In [6]:
# for item in tqdm(kb_data):
#     if item['subject_id'] not in subject:
#         subject[item['subject_id']] = len(subject)
#     for t in item['data']:
#         if t['predicate'] == '摘要':continue
#         if t['predicate'] not in predicate:
#             predicate[t['predicate']] = len(predicate)
#         if t['object'] not in type_object:
#             type_object[t['object']] = len(type_object)
#     for t_type in item['type']:
#         if t_type not in type_object:
#             type_object[t_type] = len(type_object)
# if 'type' not in predicate:
#     predicate['type'] = len(predicate)

In [7]:
# spo = []
# for item in tqdm(kb_data):
#     subject_id = subject[item['subject_id']]
#     for t_type in item['type']:
#         spo.append([subject_id,predicate['type'],type_object[t_type]])
#     for p in item['data']:
#         if p['predicate'] == '摘要':continue
#         spo.append([subject_id,predicate[p['predicate']],type_object[p['object']]])

In [8]:
# with open('entity_embedding/train2id.txt','w') as f:
#     #0 1 0
#     f.write(str(len(spo))+'\n')
#     for spo_line in spo:
#         f.write(str(spo_line[0])+' '+str(spo_line[1])+' '+str(spo_line[2])+'\n')

In [9]:
# # 14951
# # /m/027rn	0
# with open('entity_embedding/entity2id.txt','w') as f:
#     f.write(str(len(type_object))+'\n')
#     for key in type_object:
#         f.write(str(key)+' '+str(type_object[key])+'\n')

In [10]:
# with open('entity_embedding/relation2id.txt','w') as f:
#     f.write(str(len(predicate))+'\n')
#     for key in predicate:
#         f.write(str(key)+' '+str(predicate[key])+'\n')

In [11]:
# gensim method

In [12]:
# jieba加字典
for item in tqdm(kb_data):
    jieba.add_word(item['subject'],freq=10000)
    for name in item['alias']:
        jieba.add_word(name, freq=10000)

  0%|          | 0/399252 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.588 seconds.
Prefix dict has been built succesfully.
100%|██████████| 399252/399252 [00:03<00:00, 114896.47it/s]


In [13]:


sentence = []
for item in tqdm(kb_data):
    sentence.append([item['subject_id'],item['type'][0]])
    for t in item['data']:
        temp = []
        if t['predicate'] == '摘要':
            abstract = jieba.lcut(t['object'])
            for word in abstract:
                if word !=item['subject']:
                    temp.append(word)
                else:
                    temp.append(item['subject_id'])
            sentence.append(temp)
        else:
            temp.append(item['subject_id'])
            temp.append(t['predicate'])
            for w in jieba.lcut(t['object']):
                temp.append(w)
            sentence.append(temp)

100%|██████████| 399252/399252 [03:57<00:00, 1683.55it/s]


In [14]:
train_data = []
with open('data/raw_data/train.json','r') as f:
    for line in f:
        sentence.append(jieba.lcut(eval(str(json.loads(line)).lower())['text']))
with open('data/raw_data/develop.json','r') as f:
    for line in f:
        sentence.append(jieba.lcut(eval(str(json.loads(line)).lower())['text']))

In [15]:
model = word2vec.Word2Vec(sentence, size=100, window=5, min_count=1, workers=10)
# 打印单词'good'的词向量
print(model.wv.word_vec('good'))
# 打印和'good'相似的前2个单词
print(model.wv.most_similar('good', topn=2))
# 保存模型到文件
model.save('embedding/w2v.model')

[-1.06740022e+00 -1.30582309e+00 -6.24896049e-01  2.74170566e+00
 -1.32954881e-01 -7.22744644e-01  9.90687966e-01  2.22109914e+00
 -6.25711977e-01  6.69428349e-01  2.50128895e-01  2.34205270e+00
 -9.85711932e-01  1.75330555e+00  1.75123736e-02 -6.24078929e-01
  5.50190394e-04 -1.19829345e+00  1.71348405e+00  6.60451233e-01
 -7.05054700e-01 -2.52027655e+00 -2.49819350e+00 -9.74730909e-01
  1.05229843e+00 -1.58508432e+00  7.35398948e-01  3.09939623e+00
 -1.80983472e+00 -2.25609541e+00  1.69330287e+00  1.97928935e-01
 -2.87188470e-01 -1.55438244e+00 -2.25666419e-01 -1.64868927e+00
 -2.28676736e-01  1.82918414e-01 -2.08738875e-02  1.31223845e+00
  9.26504910e-01 -4.16131198e-01 -4.47655112e-01 -6.56702995e-01
 -3.93060774e-01 -2.33329311e-01 -9.28635970e-02 -7.27430761e-01
  5.80050230e-01  6.67945683e-01  6.72419012e-01 -1.15615800e-01
 -2.58462161e-01  1.58006990e+00  8.96789134e-01 -1.67929089e+00
 -2.91498989e-01 -1.08625698e+00  1.92022491e+00  1.66703090e-01
 -1.00618199e-01 -2.27684

/home/zhukaihua/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [16]:
from gensim.models import Word2Vec

entity_embedding = Word2Vec.load('embedding/w2v.model')
print(entity_embedding.wv.word_vec('搜索'))

/home/zhukaihua/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


[ 0.83644915 -0.13179712 -0.32689977  1.4807096   0.26997417 -0.7888522
  0.18786314 -1.0655216   0.15017082  0.7930093   0.6204069  -0.12830432
  0.24433438  0.85823154  0.02746648  0.5008542  -0.23827039  0.58007616
  1.7565713  -1.0904726   1.9679656  -1.5737779   1.3745987   0.76041174
  0.19700734  2.0845032   0.71697253 -0.81742346 -0.75465655  0.43983054
 -0.91081196 -0.12020243  2.330965   -0.9543566  -1.795786   -1.2433215
  0.06511574 -1.0525746   1.1140516  -0.2883513   1.0940838   0.75270617
  0.4522416  -0.5509725  -1.1325698  -1.7036166  -1.9608523  -0.38575265
  0.5976845   1.623124   -0.4178196   0.6338049  -0.9562957   1.2905524
  0.28920326 -1.1495526   0.08914449  0.2507475  -1.615023    0.34573108
 -0.08599114  1.1669557   0.39803565 -0.66602045 -0.05413941  0.61265546
  1.4831152   2.579434   -0.7199152  -0.49818987 -0.39623013 -0.8251795
  1.5805804  -2.870432   -0.7189991  -1.184574   -0.6879349  -1.5456197
 -0.9740356  -0.7846144  -0.40417776  0.4827905   0.9376